In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
from ast import literal_eval
from collections import defaultdict
from tqdm import tqdm

In [15]:
data = pd.read_csv('./data/agg_df_with_full_text.csv', converters={
    "users": literal_eval,
    "scores": literal_eval
    })
# data = pd.read_csv('./data/agg_df.csv', converters={
#     "users": literal_eval,
#     "scores": literal_eval
#     })
rating_key = 'Readability and newsworthiness:::The science news story should be published in the news'
dimension_to_id = {dim: j for j,dim in enumerate(set(data['dimension']))}
annotator_set = set()
item_set = set()
for j,row in data.iterrows():
    if row['dimension'] == rating_key and len(row['users']) > 1:
        annotator_set.update(set(row['users']))
        item_set.add(row['instance_id'])
#annotator_set = set([id_ for j,row in data.iterrows() for id_ in row['users'] if row['dimension'] == rating_key and len(row['users']) > 1])
ann_to_id = {ann: j for j,ann in enumerate(annotator_set)}
instance_to_row = {id_: j for j,id_ in enumerate(item_set)}
score_vector_set = {}
item_to_ann = defaultdict(list)
ann_to_item = defaultdict(list)
singletons = {}
for j,row in data.iterrows():
    if row['dimension'] == rating_key:
        if len(row['users']) > 1:
            for ann,score in zip(row['users'], row['scores']):
                score_vector_set[(instance_to_row[row['instance_id']],ann_to_id[ann])] = score
                item_to_ann[instance_to_row[row['instance_id']]].append(ann_to_id[ann])
                ann_to_item[ann_to_id[ann]].append(instance_to_row[row['instance_id']])
        elif len(row['users']) == 1:
            singletons[row['instance_id']] = score

In [65]:
# Get initial parameters for item mean, var and ann mean,var
item_mu = np.array([0.]*len(instance_to_row))
item_sig = np.array([1.]*len(instance_to_row))

ann_mu = np.array([0.]*len(ann_to_id))
ann_sig = np.array([1.]*len(ann_to_id))

# Global scale parameter
scale = np.array([1.])

In [66]:
# Likelihood of data fit
def py(item_mu, item_var, i):
    return np.random.normal(loc=item_mu[i], scale=item_var[i])

def px(item_mu, item_var, ann_mu, ann_var, i, j):
    # Sample from y
    yi = np.random.normal(loc=item_mu[i], scale=item_var[i])
    # Sample from a
    aj = np.random.normal(loc=item_mu[i], scale=item_var[i])

    #Likelihood
    mu = yi + aj
    return np.random.normal(loc=mu, scale=scale)

def norm_pdf(x, mu, sig):
    return (1 / np.sqrt(2*np.pi*sig**2)) * np.exp(-0.5*((x - mu) / sig)**2)

def log_norm_pdf(x, mu, sig):
    return (-0.5 * np.log(2*np.pi*sig**2)) - 0.5*((x - mu) / sig)**2

def d_norm_pdf(x, mu, sig):
    return (x - mu) / sig**2 * -norm_pdf(x, mu, sig)

def max_i(observations, item_mu, ann_mu):
    # Collect x_ij, mu_i, and mu_j
    ratings = defaultdict(list)
    for obs in observations:
        ratings[obs[0]].append(ann_mu[obs[1]] - observations[obs])
    for i in ratings:
        ratings[i] = np.mean(r)

    return np.array([ratings[i] for i in range(len(instance_to_row))])

def max_j(observations, item_mu, ann_mu):
    # Collect x_ij, mu_i, and mu_j
    ratings = defaultdict(list)
    for obs in observations:
        ratings[obs[1]].append(item_mu[obs[0]] - observations[obs])
    for j in ratings:
        ratings[j] = np.mean(r)

    return np.array([ratings[j] for j in range(len(ann_to_id))])

def max_sigma(observations, item_mu, ann_mu):
    return np.sqrt(np.mean([(observations[obs] - (item_mu[obs[0]] + ann_mu[obs[1]]))**2 for obs in observations]))

# def py_given_x_theta(y, x, item_mu, item_sig, ann_mu, ann_sig):
#     # Get mean of distributions:
#     top_mu = x*item_mu - item_mu*ann_mu
#     bot_mu = item_mu + ann_mu

#     # get sig of distirbutions
#     top_sig = x*item_sig - ann_mu*item_sig + item_mu*ann_sig
#     bot_sig = ann_sig

#     print(top_mu, bot_mu, top_sig, bot_sig)
#     # Calculate values
#     return norm_pdf(y, top_mu, top_sig) / norm_pdf(y, bot_mu, bot_sig)


def py_given_x_theta(y, x, item_mu, item_sig, ann_mu, ann_sig):
    # Get mean of distributions:
    num = norm_pdf(x, ann_mu + y, ann_sig) * norm_pdf(y, item_mu, ann_sig)
    den = norm_pdf(x, ann_mu + item_mu, ann_sig)
    # Calculate values
    return num / den


def loglikelihood(observations, item_mu, ann_mu, sig):
    N = len(observations)
    term1 = -N / 2 * np.log(2*np.pi)
    term2 = -N / 2 * np.log(sig**2)
    term3 = (-1 / (2*sig**2)) * np.sum([(observations[obs] - (item_mu[obs[0]] + ann_mu[obs[1]]))**2 for obs in observations])

    return term1 + term2 + term2
    # #Factorizes based on obersvations, for continuous rvs the likelihood is the PDF evaluated at X
    # ll = 0.
    # for obs in observations:
    #     x = observations[obs]
    #     ll += log_norm_pdf(x, item_mu[obs[0]] + ann_mu[obs[1]], sig)
    # return ll

In [ ]:
py_given_x_theta(-5, 3.0, item_mu[0], item_sig[0], ann_mu[694], ann_sig[694])

In [ ]:
loglikelihood(score_vector_set, item_mu, ann_mu, 1.)

In [ ]:
len(instance_to_row)

In [ ]:
# Get initial parameters for item mean, var and ann mean,var
item_mu = np.array([0.]*len(instance_to_row))
item_var = np.array([1.]*len(instance_to_row))

ann_mu = np.array([0.]*len(ann_to_id))
ann_var = np.array([1.]*len(ann_to_id))

# Global scale parameter
scale = np.array([1.])

for i in range(100):
    print(loglikelihood(score_vector_set, item_mu, ann_mu, scale))
    item_mu = max_i(score_vector_set, item_mu, ann_mu)
    ann_mu = max_j(score_vector_set, item_mu, ann_mu)
    scale = max_sigma(score_vector_set, item_mu, ann_mu)
    


In [ ]:
loglikelihood(score_vector_set, new_item, new_ann, new_sigma)

In [ ]:
ratings = [[]]*item_mu.shape[0]
for obs in score_vector_set:
    ratings[obs[0]].append(ann_mu[obs[1]] - score_vector_set[obs])
# for i,r in enumerate(ratings):
#     ratings[i] = np.array(r).mean()

# np.array(ratings)

In [ ]:
ratings = [[]]*item_mu.shape[0]

In [ ]:
len(ratings)

## Try as a Gaussian Mixture model

Note that since we are using a lot of Gaussians and the data is sparse, this is prone to singularities (variance goes to 0)

See here for some solutions: https://stats.stackexchange.com/questions/219302/singularity-issues-in-gaussian-mixture-model

Ideally we will use MAP, for now just reset the mean and variance when singularities occur (and then we can just set them to their proper means later)

In [18]:
from scipy import stats
import ipdb
eps = 1e-8
def norm_pdf(x, mu, sig):
    return stats.norm.pdf(x, mu, sig)
    #return (1 / np.sqrt(2*np.pi*sig**2)) * np.exp(-0.5*((x - mu) / sig)**2)
    
def lognorm_pdf(x, mu, sig):
    return -0.5*(np.log(2*np.pi*sig**2) + ((x - mu) / sig)**2)

def rijk(x, item_mu, item_sig, ann_mu, ann_sig, p):
    term1 = max(p * np.exp(lognorm_pdf(x, item_mu, item_sig)), eps)
    term2 = max((1. - p) * np.exp(lognorm_pdf(x, ann_mu, ann_sig)), eps)
    
#     if np.isnan(term1 / (term1 + term2)) or np.isnan(term2 / (term1 + term2)):
#         ipdb.set_trace()
    
    return [term1 / (term1 + term2), term2 / (term1 + term2)]

def e_step(observations, item_mu, item_var, ann_mu, ann_var, p):
    rij = defaultdict(lambda: [0.,0.])
    for obs in observations:
        rij[obs] = rijk(observations[obs], item_mu[obs[0]], item_sig[obs[0]], ann_mu[obs[1]], ann_sig[obs[1]], p[obs[1]])
    return rij

def m_step(observations, item_mu, ann_mu, rij):
    new_mu_i = np.array([0.0]*len(instance_to_row))
    new_var_i = np.array([1.0]*len(instance_to_row))

    new_mu_j = np.array([0.0]*len(ann_to_id))
    new_var_j = np.array([1.0]*len(ann_to_id))

    new_p = np.array([0.0]*len(ann_to_id))

    # Get new i
    for i in range(len(instance_to_row)):
        r = np.array([rij[(i,j)][0] for j in item_to_ann[i]])
        x = np.array([observations[(i,j)] for j in item_to_ann[i]])
        new_mu_i[i] = (r*x).sum() / r.sum()
        new_var_i[i] = (r * ((x-new_mu_i[i])**2)).sum() / r.sum()
        
        if new_var_i[i] <= eps or np.isnan(new_var_i[i]):
            # singularity, reset
            new_mu_i[i] = np.random.uniform(low=0.0, high=4.0)
            new_var_i[i] = np.random.uniform(low=1.0, high=3.0)
        
        if np.isnan(new_mu_i[i]) or np.isnan(new_var_i[i]):
            ipdb.set_trace()
            pass

    for j in range(len(ann_to_id)):
        # Using rij[1] gives 1 - p
        new_p[j] = 1 - r.mean()
        r = np.array([rij[(i,j)][1] for i in ann_to_item[j]])
        x = np.array([observations[(i,j)] for i in ann_to_item[j]])
        new_mu_j[j] = (r*x).sum() / r.sum()
        new_var_j[j] = (r * ((x-new_mu_j[j])**2)).sum() / r.sum()
        
        if new_var_j[j] <= eps or np.isnan(new_var_j[j]):
            # singularity, reset
            new_mu_j[j] = np.random.uniform(low=0.0, high=4.0)
            new_var_j[j] = np.random.uniform(low=1.0, high=3.0)
            new_p[j] = 0.5
        
        if np.isnan(new_mu_j[j]) or np.isnan(new_var_j[j]) or np.isnan(new_p[j]):
            ipdb.set_trace()
            pass

    return new_mu_i, new_var_i, new_mu_j, new_var_j, new_p

def loglikelihood(observations, item_mu, item_sig, ann_mu, ann_sig, p):
    return np.array([np.log(p[obs[1]]) + lognorm_pdf(observations[obs], item_mu[obs[0]], item_sig[obs[0]]) + np.log((1-p[obs[1]])) 
            + lognorm_pdf(observations[obs], ann_mu[obs[1]], ann_sig[obs[1]]) for obs in observations])

In [19]:
# Get initial parameters for item mean, var and ann mean,var
item_mu = np.random.uniform(low=0.0, high=4.0, size=len(instance_to_row))#np.array([2.]*len(instance_to_row))
item_sig = np.random.uniform(low=1.0, high=3.0, size=len(instance_to_row))# np.array([1.]*len(instance_to_row))

ann_mu = np.random.uniform(low=0.0, high=4.0, size=len(ann_to_id))#np.array([2.]*len(ann_to_id))
ann_sig = np.random.uniform(low=1.0, high=3.0, size=len(ann_to_id))#np.array([1.]*len(ann_to_id))

p = np.array([0.5]*len(ann_to_id))

ll = loglikelihood(score_vector_set, item_mu, item_sig, ann_mu, ann_sig, p)

with tqdm(range(200)) as pbar:
    for n in pbar:
        rij = e_step(score_vector_set, item_mu, item_sig, ann_mu, ann_sig, p)
        item_mu_new, item_sig_new, ann_mu_new, ann_sig_new, p_new = m_step(score_vector_set, item_mu, ann_mu, rij)
        ll_new_all = loglikelihood(score_vector_set, item_mu_new, item_sig_new, ann_mu_new, ann_sig_new, p_new)
        ll_new = ll_new_all.sum()
        pbar.set_description(f"LL: {ll_new}")
    #     if abs(ll_new - ll) < 0.01:
    #         break
        ll = ll_new
        item_mu = item_mu_new
        item_sig = item_sig_new
        ann_mu = ann_mu_new
        ann_sig = ann_sig_new
        p = p_new

LL: -1.1114188006531645e+18:   8%|██████████                                                                                                            | 17/200 [00:02<00:24,  7.48it/s]


KeyboardInterrupt: 

In [19]:
ll_new_all.max()

31.482863325554483

In [125]:
list(score_vector_set.keys())[5804]

(1020, 977)

In [140]:
print(np.exp(lognorm_pdf(list(score_vector_set.values())[5804], item_mu[1020], item_sig[1020])))
print(lognorm_pdf(list(score_vector_set.values())[5804], ann_mu[977], ann_sig[977]))

39894228.04014329
17.501742210747693


In [131]:
print(norm_pdf(list(score_vector_set.values())[5804], item_mu[1020], item_sig[1020]))
print(norm_pdf(list(score_vector_set.values())[5804], ann_mu[977], ann_sig[977]))

39894228.04014323
39894228.04014323


In [132]:
list(score_vector_set.values())[5804]

3.0

In [139]:
ann_mu[977]

3.0000000000000004

In [137]:
[score_vector_set[(i,977)] for i in ann_to_item[977]]

[3.0, 4.0, 4.0, 3.0, 3.0]

In [10]:
all_mu = [np.mean([score_vector_set[(i,j)] for j in item_to_ann[i]]) for i in range(len(instance_to_row))]
abs(item_mu - all_mu).mean()

0.457046178500062

In [110]:
max(item_sig)

3.999999929278521

In [111]:
p

array([0.44195984, 0.25000018, 0.79696567, ..., 0.4       , 0.79999997,
       0.6       ])

In [ ]:
item_sig

In [ ]:
item_mu[5]

In [ ]:
np.mean([score_vector_set[obs] for obs in score_vector_set if obs[0] == 5])

In [ ]:
[score_vector_set[obs] for obs in score_vector_set if obs[0] == 5]

In [ ]:
item_to_ann[5]

In [ ]:
ann_mu[245]

In [ ]:
np.argmin(p)

In [ ]:
([score_vector_set[obs] for obs in score_vector_set if obs[1] == 245])

In [ ]:
ann_to_item[245]

In [ ]:
item_sig[1390]

In [ ]:
[score_vector_set[obs] for obs in score_vector_set if obs[0] == 1390]

In [ ]:
ann_sig[498]

In [ ]:
ann_to_item[1130]

In [ ]:
base_averages = []
for i in range(len(instance_to_row)):
    x = np.array([score_vector_set[(i,j)] for j in item_to_ann[i]])
    base_averages.append(x.mean())
base_averages = np.array(base_averages)

In [ ]:
abs(base_averages - item_mu).mean()

In [ ]:
item_var